In [1]:
import pathlib

import tensorflow as tf
from absl import logging
from tfx import v1 as tfx
from tfx.v1 import proto
from tfx.proto import example_gen_pb2
from tfx.v1.components import ExampleValidator
from tfx.v1.components import SchemaGen
from tfx.v1.components import StatisticsGen
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

2023-06-15 20:23:41.682399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
import pprint
pp = pprint.PrettyPrinter()
from mymllib import Utils, rawdata_duckdb_sql
from mymllib.tfx.example_gen.duckdb.component import DuckDBExampleGen
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [3]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.10.1
TFX version: 1.11.0


In [4]:
###################################################################################################################
COMPETITION = 'store-sales-time-series-forecasting'
PIPELINE_NAME = COMPETITION
########################### Set up variables ####################################################################
CURRENT_DIR = globals()['_dh'][0] if '_dh' in globals() else pathlib.Path(__file__).parent.as_posix()
DATA_DIR = pathlib.Path(CURRENT_DIR).joinpath("data")
TFX_DIR = pathlib.Path(CURRENT_DIR).joinpath("out/tfx").joinpath(PIPELINE_NAME)
# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = TFX_DIR.joinpath('pipelines')
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = TFX_DIR.joinpath('metadata')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = TFX_DIR.joinpath('serving_model')
###########################Set up variables#####################################################################
print("Data dir: %s" % DATA_DIR)
print("tfx dir: %s" % TFX_DIR)
print("Pipelines root: %s" % PIPELINE_ROOT)

Data dir: /Users/ismailsimsek/development/StoreSalesTimeSeriesForecasting/data
tfx dir: /Users/ismailsimsek/development/StoreSalesTimeSeriesForecasting/out/tfx/store-sales-time-series-forecasting
Pipelines root: /Users/ismailsimsek/development/StoreSalesTimeSeriesForecasting/out/tfx/store-sales-time-series-forecasting/pipelines


In [5]:
Utils.download(datadir=DATA_DIR, competition=COMPETITION)

In [6]:
# Declare the InteractiveContext and use a local sqlite file as the metadata store.
# You can ignore the warning about the missing metadata config file
context = InteractiveContext(pipeline_root=PIPELINE_ROOT.as_posix())

In [7]:
# NOTE! query inputs are using following config by default
# ```input_config = example_gen_pb2.Input(splits=[
#       example_gen_pb2.Input.Split(name='single_split', pattern=split_pattern)
#   ])
# ```
input_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern=rawdata_duckdb_sql(data="train")),
    example_gen_pb2.Input.Split(name='eval', pattern=rawdata_duckdb_sql(data="test")),
])
output = proto.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 proto.SplitConfig.Split(name='train', hash_buckets=8),
                 proto.SplitConfig.Split(name='eval', hash_buckets=2)
             ]))

# example_gen = DuckDBExampleGen(input_config=input_config)
example_gen = DuckDBExampleGen(query=rawdata_duckdb_sql(data="train"), output_config=output)

In [8]:
context.run(example_gen)

INFO:absl:Running driver for DuckDBExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for DuckDBExampleGen
INFO:absl:Generating examples.


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))


KeyboardInterrupt



In [ ]:
print(execution_result)
print(example_gen)
print('----------------------')
# Computes statistics over data for visualization and example validation.
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
print('----------------------')
print(statistics_gen)
print('----------------------')

# Generates schema based on statistics files.
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
print('----------------------')
print(schema_gen)
print('----------------------')

# Performs anomaly detection based on statistics and data schema.
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Computes statistics over data for visualization and example validation.
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
print('----------------------')
print(statistics_gen)
print('----------------------')